In [68]:
import pandas as pd
from river import stream,tree,metrics
import utils
from encoding import prefix_bin
import csv
import copy, json
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm
import pickle as pkl
import matplotlib.pyplot as plt
import os

In [69]:
dataset = stream.iter_csv(
            './bpic17_length28.csv',
            )

totallength = len(list(dataset))

In [70]:
dataset = stream.iter_csv(
            './bpic17_length28.csv',
#             drop=['Complete Timestamp'],
            )
enctype = 'Index-base'

In [71]:
key_pair = {
# 'Case ID':'caseid',
# 'Activity':'activity',
# 'Resource':'resource',
# 'Complete Timestamp':'ts',
}
catatars= ['activity','resource']

case_dict ={}
training_models ={}
feature_matrix ={}

casecount = 0
rowcounter = 0
resultdict ={}
acc_dict ={}
window_acc_dict={}
running_case = 0
prediction_result = {}
graceperiod_finish=0
finishedcases = set()
usedingrace = set()
window_size = 50

somewhere = {}
window_label_dist ={}

In [76]:
def display_progress(row_counting, total_length, interval=500):
    if rowcounter%interval == 0:
        print(round(rowcounter*100/totallength,2) ,'%', 'Case finished: %s'%(casecount))

In [73]:
for x,y in dataset:
    display_progress(rowcounter, totallength)
    rowcounter +=1
    
    utils.dictkey_chg(x, key_pair)
    # Event stream change dictionary keys
    x['ts'] = x['ts'][:-4]
    
    # Check label possible
    
    # Initialize case by prefix length
    caseid = x['caseid']    

    x.pop('caseid')
    
    case_bin = prefix_bin(caseid, x)
    
    if caseid not in list(case_dict.keys()):
        case_bin.set_prefix_length(1)    
        case_dict[caseid] = []
        
    elif caseid in finishedcases:
        pass
    
    else:
        case_bin.set_prefix_length(len(case_dict[caseid])+1)
        case_bin.set_prev_enc(case_dict[caseid][-1])
    
    # Encode event and cases and add to DB
    ts = case_bin.event['ts']
    
    case_bin.set_enctype(enctype)
    case_bin.update_encoded(catattrs=catatars,enctype=enctype)
    current_activity = [x for x in case_bin.encoded.keys() if 'activity_%s'%(case_bin.prefix_length) in x].pop()
            
    # Set current activity as outcome of previous event
    if case_bin.prefix_length != 1:
        case_bin.prev_enc.update_truelabel(current_activity)


    case_dict[caseid].append(case_bin)
    usedingrace.add(caseid)
    # Detect label appeared case 
    if x['activity'] == 'End' and caseid not in finishedcases:
        
        finishedcases.add(caseid)
        # Adding newly finished case to training set.    
        casecount +=1
        # Grace period to collect feature matrix
        if casecount <50:
            case_length = len(case_dict[caseid])
            for prefix in range(1, case_length):
                if 'prefix_%s'%(prefix+1) not in list(feature_matrix.keys()):
                    feature_matrix['prefix_%s'%(prefix+1)]=set()
                    # Initialize classifier and performance matrix and updating count
                    training_models['prefix_%s'%(prefix+1)] = [tree.ExtremelyFastDecisionTreeClassifier(grace_period=100,split_criterion='info_gain'),
                                                               metrics.Accuracy(),
                                                               0,0]
                feature_list = list(case_dict[caseid][prefix].encoded.keys())
                for x in feature_list: feature_matrix['prefix_%s'%(prefix+1)].add(x) 
            graceperiod_finish = case_dict[caseid][-1].event['ts']
            for t in training_models.keys():
                training_models[t][3] = graceperiod_finish
            somewhere[caseid] = case_dict[caseid]
            case_dict.pop(caseid)
            
        else:
            break

0.0 % Case finished: 0
6.22 % Case finished: 8
12.44 % Case finished: 24
18.67 % Case finished: 36


In [ ]:
case_dict ={}
for x,y in dataset:
    display_progress(rowcounter,totallength,3000)
    rowcounter +=1
    # Event stream change dictionary keys
    utils.dictkey_chg(x, key_pair)

    x['ts'] = x['ts'][:-4]
#     x['ts'] += ' 00:00:00' 

    # Check label possible
    
    # Initialize case by prefix length
    caseid = x['caseid']    
    x.pop('caseid')
    
    if caseid in usedingrace:
        continue
    
    case_bin = prefix_bin(caseid, x)

    if caseid not in list(case_dict.keys()):
        case_bin.set_prefix_length(1)    
        case_dict[caseid] = []
        running_case +=1
        
    elif caseid in finishedcases:
        pass
    
    else:
        case_bin.set_prefix_length(len(case_dict[caseid])+1)
        case_bin.set_prev_enc(case_dict[caseid][-1])

    # Encode event and cases and add to DB
    ts = case_bin.event['ts']

    case_bin.set_enctype(enctype)
    case_bin.update_encoded(catattrs=catatars,enctype=enctype)        
    current_activity = [x for x in case_bin.encoded.keys() if 'activity_%s'%(case_bin.prefix_length) in x].pop()

    if case_bin.prefix_length != 1:
        case_bin.prev_enc.update_truelabel(next_activity)

    if case_bin.prefix_length >=2:
        case_bin.encoded = utils.readjustment_training(case_bin.encoded, feature_matrix['prefix_%s'%(case_bin.prefix_length)])
        x_test = case_bin.encoded
        model = training_models['prefix_%s'%(case_bin.prefix_length)][0]
        y_pred = model.predict_one(x_test)
        if type(y_pred) != str:
            y_pred = 'False'
        modelid,pred_value = copy.deepcopy(training_models['prefix_%s'%(case_bin.prefix_length)][2]), copy.deepcopy(y_pred)
        case_bin.update_prediction((modelid, (pred_value,ts)))        
    case_dict[caseid].append(case_bin)

    # Detect label appeared case 
    if progress != '' and caseid not in finishedcases:
        finishedcases.add(caseid)
        # Adding newly finished case to training set.    
        casecount +=1    
        # Real training start

        # Modify encoded attributes of cases with feature matrix
        case_length = len(case_dict[caseid])

        y = next_activity
        x = case_dict[caseid][-2].encoded
        previous_event_length = case_dict[caseid][-2].prefix_length
        model = training_models['prefix_%s'%(previous_event_length)][0]
        model.learn_one(x,y)
        training_models['prefix_%s'%(previous_event_length)][2] +=1
        y_pred = model.predict_one(x)
        training_models['prefix_%s'%(previous_event_length)][1].update(y,y_pred)

        for cases in list(case_dict.keys()):
            if len(case_dict[cases]) == previous_event_length-1:
                x_test = case_dict[cases][-1].encoded
                y_pred = model.predict_one(x_test)
                modelid,pred_value = copy.deepcopy(training_models['prefix_%s'%(prefix+1)][2]), copy.deepcopy(y_pred)
                case_dict[cases][-1].update_prediction((modelid, (pred_value,ts)))
                prediction_key = str(cases)+'_'+str(prefix+1)
                if str(cases)+'_'+str(prefix+2) not in prediction_result.keys():
                    if prediction_key not in prediction_result.keys():
                        prediction_result[prediction_key] = {}
                        prediction_result[prediction_key][modelid] = (pred_value,ts)
                    else:
                        prediction_result[prediction_key][modelid] = (pred_value,ts)
#             print(case_dict[caseid][-2].predicted)
        copying = copy.deepcopy(case_dict[caseid])
        resultdict[caseid] = copying
        case_dict[caseid] =[]
        running_case -=1

        for prefix in training_models.keys():
            if prefix not in list(acc_dict.keys()):
                acc_dict[prefix]=[training_models[prefix][1].get()]
            else:
                acc_dict[prefix].append(training_models[prefix][1].get())
